In [1]:
import os

In [2]:
%pwd

'c:\\Users\\H P\\Desktop\\Machine Learning Projects\\ML Ops Project\\Chest_Cancer_Classification_Using_MLflow_-_DVC\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\H P\\Desktop\\Machine Learning Projects\\ML Ops Project\\Chest_Cancer_Classification_Using_MLflow_-_DVC'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/shabbu8111999/Chest_Cancer_Classification_Using_MLflow_-_DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "shabbu8111999"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "89fd3ee38ac225a7a6c8489337e31ea344128042"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from ChestCancerClassification.constants import *
from ChestCancerClassification.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data = "artifacts/data_ingestion/Chest_Scan_CT",
            mlflow_uri = "https://dagshub.com/shabbu8111999/Chest_Cancer_Classification_Using_MLflow_-_DVC.mlflow",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )

        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

c:\Users\H P\anaconda3\envs\cancer\lib\site-packages\pydantic\_internal\_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1. / 255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {'loss': self.score[0], 'accuracy': self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-07-14 15:31:28,693: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2025-07-14 15:31:28,723: INFO: common: yaml file: params.yaml loaded successfully:]
[2025-07-14 15:31:28,727: INFO: common: created directory at: artifacts:]
Found 102 images belonging to 2 classes.
13/13 [==============================] - 21s 2s/step - loss: 8.0002e-04 - accuracy: 1.0000


2025/07/14 15:31:52 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-07-14 15:31:55,078: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.:]
INFO:tensorflow:Assets written to: C:\Users\HP12D5~1\AppData\Local\Temp\tmpnpt9rh_n\model\data\model\assets
[2025-07-14 15:31:56,146: INFO: builder_impl: Assets written to: C:\Users\HP12D5~1\AppData\Local\Temp\tmpnpt9rh_n\model\data\model\assets:]


c:\Users\H P\anaconda3\envs\cancer\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/07/14 15:32:47 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 4
Created version '4' of model 'VGG16Model'.
